In [15]:
import nltk
import requests
import boewebconnectpy
import urllib3
import re
import nltk
from nltk.tokenize import word_tokenize
import pandas as pd

nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab')
nltk.download('punkt')


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\339755\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\339755\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\339755\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
boewebconnectpy.set_boe_config()

Successfully set web config settings for urllib, urllib3, requests


In [4]:
#session = requests.Session()

urllib3.make_headers()

r = urllib3.request("GET", "https://www.bankofengland.co.uk/stress-testing/2025/key-elements-bank-capital", headers={"User-Agent": "Mozilla/5.0"})



In [5]:
text = r.data.decode('utf-8')

text = text.replace('\n', ' ').replace('\r', '').replace('\t', '')

text = re.sub(r'<[^>]+>', '', text)

text = re.sub(r'{[^>]+}', '', text)

text = re.sub(r'\s+', ' ', text)

text

' Stress testing the UK banking system: Key elements of the 2025 Bank Capital stress test | Bank of England var CheckCookies = (),"".length>0)if(a&&"performance"in a&&a.performance&&"function"==typeof a.performance.setResourceTimingBufferSize)a.performance.setResourceTimingBufferSize();!function() Published on 24 March 2025 Browse content Contents 1: Executive summaryThe Bank of England (‘the Bank’) is carrying out a Bank Capital Stress Test in 2025. Stress testing is used by the Bank to determine the UK banking system’s ability to withstand an adverse scenario while continuing to support the real economy. By doing so the Bank aims to ensure banks can absorb rather than amplify shocks and have the capacity to continue to serve UK households and businesses.The 2025 Bank Capital Stress Test will assess the resilience of the UK banking system to deep simultaneous recessions in the UK and global economies, large falls in asset prices, higher global interest rates, and a stressed level of m

In [16]:
from nltk.tokenize import word_tokenize, sent_tokenize
sentences = sent_tokenize(text)
words = word_tokenize(text)

tagged_words = nltk.pos_tag(words)
tagged_words

[('Stress', 'NN'),
 ('testing', 'VBG'),
 ('the', 'DT'),
 ('UK', 'NNP'),
 ('banking', 'NN'),
 ('system', 'NN'),
 (':', ':'),
 ('Key', 'JJ'),
 ('elements', 'NNS'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('2025', 'CD'),
 ('Bank', 'NNP'),
 ('Capital', 'NNP'),
 ('stress', 'JJ'),
 ('test', 'NN'),
 ('|', 'NNP'),
 ('Bank', 'NNP'),
 ('of', 'IN'),
 ('England', 'NNP'),
 ('var', 'NN'),
 ('CheckCookies', 'NNP'),
 ('=', 'NNP'),
 ('(', '('),
 (')', ')'),
 (',', ','),
 ("''", "''"),
 ("''", "''"),
 ('.length', 'NNP'),
 ('>', 'NNP'),
 ('0', 'CD'),
 (')', ')'),
 ('if', 'IN'),
 ('(', '('),
 ('a', 'DT'),
 ('&', 'CC'),
 ('&', 'CC'),
 ("''", "''"),
 ('performance', 'NN'),
 ("''", "''"),
 ('in', 'IN'),
 ('a', 'DT'),
 ('&', 'CC'),
 ('&', 'CC'),
 ('a.performance', 'NN'),
 ('&', 'CC'),
 ('&', 'CC'),
 ("''", "''"),
 ('function', 'NN'),
 ("''", "''"),
 ('==typeof', 'NNP'),
 ('a.performance.setResourceTimingBufferSize', 'NN'),
 (')', ')'),
 ('a.performance.setResourceTimingBufferSize', 'NN'),
 ('(', '('),
 (')', ')'),
 (

In [11]:
sentence_data = pd.DataFrame(sentences, columns=['Sentence'])

sentence_data.head()

,Sentence
0,Stress testing the UK banking system: Key ele...
1,Stress testing is used by the Bank to determin...
2,By doing so the Bank aims to ensure banks can ...
3,"Rather, like previous concurrent stress test s..."
4,"It is not a set of events that is expected, or..."
